In [ ]:
#Martin MacDonald
#Hons project
#2025

In [ ]:
#This is a Biham-Middleton-Levine (BML) Traffic Model simulation with configurable parameters.
#East-bound cars attempt to move east, if the cell east is free then they will move east, if not possible they will stay in place.
#South-bound cars attempt to move south, if the cell south is free then they will move south, if not possible they will stay in place.
#The grid is a torus, meaning that when cars move off the edge at one side, they reappear at the other side.

#The goal of this simulation is observe traffic patterns and visualise how traffic flow depends on car density.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from PIL import Image

In [ ]:
def initialise_grid(rows, cols, density, east_dir_p, seed):
    #0 is an empty space, 1 is an east moving car, 2 is a south moving car.
    #east_dir_p is the probablility that a car will be east moving, a higher value means it is more likely to be east moving.
    rng = np.random.default_rng(seed)
    total_size = rows * cols
    num_cars = int(total_size * density)
    
    #Sets up the grid as an array of given size total with zeros in every cell.
    grid = np.zeros(total_size, dtype=np.int8)
    #Sets up each car to be either east moving or south moving using random number generation.
    cars = rng.choice([1,2], size=num_cars, p=[east_dir_p, 1 - east_dir_p])
    #Sets the positions of car into an array which is determined by random number generation.
    positions = rng.choice(total_size, size=num_cars, replace=False)
    #Inserts each car into its given position within the grid.
    grid[positions]=cars
    #Returns the populated grid which has been reshaped into a 2D array with sides equal to rows and cols.
    return grid.reshape((rows, cols))

In [ ]:
#When stepping through the simulation, east moving cars will have priority, this prevents cars moving in different directions from trying to occupy the same empty space at the same time.

In [ ]:
def step_grid(grid):
    #Used to keep track of how many cars have moved this step
    cars_moved = 0

    #East moving cars
    
    #Assign east and empty_cell to their corresponding values within the grid
    east_cars = grid == 1
    empty_cell = grid == 0

    #Determines whether an east-bound car can move or not by identifying whether the next cell east is empty or not.
    movable = east_cars & np.roll(empty_cell, -1, axis=1)

    #Saves the cars origin point before it's moved
    car_origin = movable
    #Saves the position which the car will move to
    targets = np.roll(movable, 1, axis=1)

    #Sets the cars origin to empty and moves the car to its new cell.
    grid[car_origin] = 0
    grid[targets] = 1

    #Add the amount of east bound-cars moved this step to the total moved.
    cars_moved += np.count_nonzero(car_origin)

    #South moving cars
    
    #Assign south_cars and empty_cell to their corresponding values within the grid
    #Assigning empty_cell again may seem redundant but after east-bound cars have moved, empty cells will all be different.
    south_cars = grid == 2
    empty_cell = grid == 0

    #Determines whether an south bound car can move or not by identifying whether the next cell south is empty or not.
    movable = south_cars & np.roll(empty_cell, -1, axis=0)

    #Saves the cars origin point before it's moved
    car_origin = movable
    #Saves the position which the car will move to
    targets = np.roll(movable, 1, axis=0)

    #Sets the cars origin to empty and moves the car to its new cell.
    grid[car_origin] = 0
    grid[targets] = 2

    #Add the amount of south-bound cars moved this step to the total moved.
    cars_moved += np.count_nonzero(car_origin)

    #Return the total number of cars moved
    return cars_moved

In [ ]:
def run_sim(rows = 50, cols = 50, density = 0.3, steps = 50, east_dir_p = 0.5, seed = None, fps = 10, gif_name = "bml_sim.gif"):
    #Initialise the grid with rows, columns, car density, east-moving car probability and seed.
    grid = initialise_grid(rows, cols, density, east_dir_p, seed)
    #Store copies of the grid which will be used to create the gif.
    frames = [grid.copy()]
    #Defines the colours needed for plotting the grid: White = empty, red = east-bound cars, blue = south-bound cars.
    cmap = ListedColormap(['white', 'red', 'blue'])

    #Keep track of the number of cars that are moved during each step.
    moved_counts = []

    #Run the simulation for however many steps the user wants.
    for t in range(steps):
        #Advance the simulation one step and save how many cars were moved.
        moved = step_grid(grid)
        #Save the amount of cars moved this step to the moved_counts array.
        moved_counts.append(moved)
        #Save the current state of the grid.
        frames.append(grid.copy())

    #After the simulation is complete
    #Print how many cars were moved at each step of the simulation.
    print(moved_counts)

    #Create gif
    gif_path=gif_name
    imgs = []
    fig, ax = plt.subplots(figsize=(4, 4))
    fig.subplots_adjust(left=0, right=1, top=1, bottom=0) 
    #Convert each frame into an image.
    for frame in frames:
        ax.clear()
        ax.imshow(frame, interpolation="nearest", cmap=cmap) 
        ax.set_xticks([])
        ax.set_yticks([])
        fig.canvas.draw()
    
        #Convert rendered frame into a NumPy array
        buf = np.frombuffer(fig.canvas.renderer.buffer_rgba(), dtype=np.uint8)
        w, h = fig.canvas.get_width_height()
        buf = buf.reshape((h, w, 4)) 
        buf = buf[:, :, :3] 

        #Convert that NumPy array to an image.
        imgs.append(Image.fromarray(buf))
    #Close the figure.
    plt.close(fig)
    
    #Save all frames as a Gif
    imgs[0].save(
        gif_path,
        save_all=True,
        append_images=imgs[1:],
        duration=int(1000 / fps),
        loop=0
    )

    #Confirmation that the gif was saved.
    print(f"Saved animation to {gif_path}")

In [ ]:
#Run the simulation with rows, columns, car density, simulation steps, east-moving car probability, seed, gif FPS, and gif name
#In the gif, white = empty cell, red = east-bound car, blue = south-bound car.
run_sim(100, 100, 0.6, 1000, 0.5, None, 1000)